Step 1: Loading Data

In [3]:
import io
import re
from datetime import datetime
import pandas as pd
import streamlit as st
from pypdf import PdfReader
import pdfplumber
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import fitz  # PyMuPDF

RuntimeError: Directory 'static/' does not exist

In [4]:
file_path = '/DraftMLResumeProject/parsed_resumes.csv'
try:
    df = pd.read_csv(file_path)
except:
    df = pd.read_csv("parsed_resumes.csv")      # manual read

In [5]:
# Print the shape of the dataset

print(f"There are {df.shape[0]} rows and {df.shape[1]} columns.")

There are 4 rows and 18 columns.


In [6]:
# Display first 5 rows of Raw Data
display(df.head())

,file,pages,used_ocr,name_guess,emails,phones,linkedin,github,other_links,skills_snippet,education_snippet,char_count,hs_gpa,degree,university,programming_languages,internships_found,graduation_date
0,Anthony Franzino Resume 2025.pdf,1,False,leverage AI.,"ajfranzino1@gmail.com, jdsaliba14@gmail.com, p...","687695376, 863) 224-1745, 863) 255-2674, 863) ...",https://www.linkedin.com/in/anthony-franzino-6...,NaN,NaN,NaN,EDUCATION AUGUST 2022 - PRESENT (Graduating Fa...,2025,NaN,NaN,"UNIVERSITY OF FLORIDA, GAINESVILLE, FL",Typescript,NaN,NaN
1,Elliot Blain Computer Science Resume.pdf,2,False,"Bachelor of Science, Computer Science",njblain1@gmail.com,941) 416-7722,NaN,NaN,https://njblain1.wixsite.com/elliotblain,NaN,"Education Bachelor of Science, Computer Scienc...",3025,NaN,Bachelor of Science,"Gainesville, FL | njblain1@gmail.com | (941) 4...","C++, Java, Matlab, Python, Sql",NaN,NaN
2,Christian Schneider Resume.pdf,1,False,Efficiently manage multiple tables,Cdschneider27@gmail.com,"407)-782-8481, 734375347\n2255",https://www.linkedin.com/in/christian-schneide...,NaN,NaN,NaN,"Education Bachelor of Science, Computer Scienc...",1610,NaN,Bachelor of Science,llege Board SAT Participant,"C++, Html, Java, Javascript",NaN,NaN
3,Lana Daraiseh TEST Resume.pdf,1,False,Project: Image Processing,lanadaraiseh@gmail.com,+966549842551,NaN,NaN,NaN,•,EDUCATION University of Florida (UF) Gainesvil...,2537,3.8/4.0,Master,"hahran, KSA, 31311","C++, Java, Python",NaN,May 2025


In [7]:
# Information about Dataset
display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   file                   4 non-null      object 
 1   pages                  4 non-null      int64  
 2   used_ocr               4 non-null      bool   
 3   name_guess             4 non-null      object 
 4   emails                 4 non-null      object 
 5   phones                 4 non-null      object 
 6   linkedin               2 non-null      object 
 7   github                 0 non-null      float64
 8   other_links            1 non-null      object 
 9   skills_snippet         1 non-null      object 
 10  education_snippet      4 non-null      object 
 11  char_count             4 non-null      int64  
 12  hs_gpa                 1 non-null      object 
 13  degree                 3 non-null      object 
 14  university             4 non-null      object 
 15  programmin

None

In [8]:
# Print Summary Statistics
display(df.describe(include = "all"))

,file,pages,used_ocr,name_guess,emails,phones,linkedin,github,other_links,skills_snippet,education_snippet,char_count,hs_gpa,degree,university,programming_languages,internships_found,graduation_date
count,4,4.00,4,4,4,4,2,0.0,1,1,4,4.000000,1,3,4,4,0.0,1
unique,4,NaN,1,4,4,4,2,NaN,1,1,4,NaN,1,2,4,4,NaN,1
top,Anthony Franzino Resume 2025.pdf,NaN,False,leverage AI.,"ajfranzino1@gmail.com, jdsaliba14@gmail.com, p...","687695376, 863) 224-1745, 863) 255-2674, 863) ...",https://www.linkedin.com/in/anthony-franzino-6...,NaN,https://njblain1.wixsite.com/elliotblain,•,EDUCATION AUGUST 2022 - PRESENT (Graduating Fa...,NaN,3.8/4.0,Bachelor of Science,"UNIVERSITY OF FLORIDA, GAINESVILLE, FL",Typescript,NaN,May 2025
freq,1,NaN,4,1,1,1,1,NaN,1,1,1,NaN,1,2,1,1,NaN,1
mean,NaN,1.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2299.250000,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,0.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,614.686031,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1610.000000,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1921.250000,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2281.000000,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,1.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2659.000000,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Count duplicates and missing values
num_duplicates = df.duplicated().sum()
missing_values = df.isnull().sum()

# Display the results
print(f"Number of duplicate rows: {num_duplicates}")
print("\nMissing values per column:")
print(missing_values)

Number of duplicate rows: 0

Missing values per column:
file                     0
pages                    0
used_ocr                 0
name_guess               0
emails                   0
phones                   0
linkedin                 2
github                   4
other_links              3
skills_snippet           3
education_snippet        0
char_count               0
hs_gpa                   3
degree                   1
university               0
programming_languages    0
internships_found        4
graduation_date          3
dtype: int64


In [10]:
# Handle missing values
df.dropna(inplace = True)

In [11]:
# Display Columns names
df.columns

Index(['file', 'pages', 'used_ocr', 'name_guess', 'emails', 'phones',
       'linkedin', 'github', 'other_links', 'skills_snippet',
       'education_snippet', 'char_count', 'hs_gpa', 'degree', 'university',
       'programming_languages', 'internships_found', 'graduation_date'],
      dtype='object')

In [12]:
# Print the shape of the dataset

print(f"There are {df.shape[0]} rows and {df.shape[1]} columns.")

There are 0 rows and 18 columns.


Step 2: Exploratory Data Analysis

In [13]:
# need column for if person got an internship or not
# code would be like internship_counts = df['internship'].value_counts()

Step 3: Preprocessing

Splitting Train/Test Data

In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer

#load data
df = pd.read_csv("resumes_with_labels.csv")
#only keep text and label for this
df = df[["text", "got_internship"]].dropna()

#train test split
X_train, X_test, y_train, y_test = train_test_split(
    df["text"], df["got_internship"], test_size = 0.2, random_state = 42, stratify = df["got_internship"]
)

#vectorize text
vectorizer = TfidfVectorizer(stop_words = "english", sublinear_tf = True, max_features = 5000, ngram_range = (1, 2), min_df = 3)
X_train_vector = vectorizer.fit_transform(X_train)
X_test_vector = vectorizer.transform(X_test)

In [15]:
display(X_train_vector)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 18360 stored elements and shape (129, 2078)>

In [16]:
display(X_test_vector)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 4276 stored elements and shape (33, 2078)>

In [17]:
X_train_vector.shape

(129, 2078)

In [18]:
# Lastly, create a second, smaller feature space using RFE
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
estimator = LogisticRegression(max_iter=1000)
rfe = RFE(estimator, n_features_to_select=10)  # Select top 10 features
rfe.fit(X_train_vector, y_train)
RFE_X_train_vector = rfe.transform(X_train_vector)
RFE_X_test_vector = rfe.transform(X_test_vector)

In [ ]:
# Neural Network
# Due to constraints related to training time and processing power, we will stick to only 3 hidden layers.
# We will need to import Keras.
#import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import callbacks
from tensorflow.keras.layers import Layer, Dense, BatchNormalization, Input
# import time for measuring training time
import time

In [19]:
X_train_vector.shape

(129, 2078)

In [20]:
RFE_X_train_vector.shape

(129, 10)

In [21]:
y_train.shape

(129,)

Neural Network

In [22]:
# Regular Model
nn_model = Sequential([
    Input(shape = (X_train.shape[1],)),
    Dense(64, activation = 'relu'),
    Dense(128, activation = 'relu'),
    Dense(128, activation = 'relu'),
    BatchNormalization(),
    Dense(64, activation = 'relu'),
    Dense(1, activation = 'sigmoid')
])
nn_model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

NameError: name 'Sequential' is not defined

In [ ]:
# Reduced Feature Space Model
nn_RFE_model = Sequential([
    Input(shape = (RFE_X_train_vector.shape[1],)),
    Dense(64, activation = 'relu'),
    Dense(128, activation = 'relu'),
    Dense(128, activation = 'relu'),
    BatchNormalization(),
    Dense(64, activation = 'relu'),
    Dense(1, activation = 'sigmoid')
])
nn_RFE_model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
es_callback = callbacks.EarlyStopping(monitor='val_loss', patience=10) # early stopping callback, tracking val_loss

In [ ]:
# Train the regular model
start_time = time.time()
nn_model.fit(X_train, y_train, epochs = 100, batch_size = 64, validation_split = 0.10, callbacks=[es_callback])
end_time = time.time()
nn_train_time = end_time - start_time
print(f"Training time: {end_time - start_time} seconds")

Neural Network Performance

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay, classification_report

In [ ]:
# Make predictions on the test data
pred_nn = nn_model.predict(X_test)
pred_nn_RFE = nn_RFE_model.predict(RFE_X_test_vector)

# Convert probabilities to binary predictions (neural network returns a probability)
pred_nn = (pred_nn > 0.5).astype(int)
pred_nn_RFE = (pred_nn_RFE > 0.5).astype(int)

# Evaluate the model using Accuracy, Precision, Recall, and F1 Score
print('Evaluating Neural Network Model \n')

accuracy_nn = accuracy_score(y_test, pred_nn)
display(f"Model accuracy: {accuracy_nn}")

precision_nn = precision_score(y_test, pred_nn)
display(f"Precision: {precision_nn:.3f}")

recall_nn = recall_score(y_test, pred_nn, average="weighted")
display(f"Recall: {recall_nn:.4f}")

f1_nn = f1_score(y_test, pred_nn, average="weighted")
display(f"f1: {f1_nn:.4f}")

# Confusion Matrix for neural network without RFE
conf_matrix = confusion_matrix(y_test, pred_nn)
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=[0, 1])
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix - Neural Network")
plt.show()

print(conf_matrix)

# Generate classification report
print(classification_report(y_test, pred_nn))

In [ ]:
# Display performance for neural network using RFE
accuracy_nn_RFE = accuracy_score(y_test, pred_nn_RFE)
display(f"Model accuracy: {accuracy_nn_RFE}")

precision_nn_RFE = precision_score(y_test, pred_nn_RFE)
display(f"Precision: {precision_nn_RFE:.3f}")

recall_nn_RFE = recall_score(y_test, pred_nn_RFE, average="weighted")
display(f"Recall: {recall_nn_RFE:.4f}")

f1_nn_RFE = f1_score(y_test, pred_nn_RFE, average="weighted")
display(f"f1: {f1_nn_RFE:.4f}")

# Confusion Matrix for neural network without RFE
conf_matrix = confusion_matrix(y_test, pred_nn_RFE)
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=[0, 1])
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix - Neural Network with RFE")
plt.show()

print(conf_matrix)

# Generate classification report
print(classification_report(y_test, pred_nn_RFE))